Librerías

In [14]:
import pandas as pd
import string
import numpy as np
import re


data = pd.read_excel('../downloads/emitidos_octubre.xlsx', header=0, skiprows=1)

Transformación de reporte afip DE VENTAS

In [2]:
#TIPO DE FACTURA
data['codigo_fc'] = [int(re.match(r'(\d+)', x).group(1)) for x in data['Tipo']]
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

data['tipo2_new'] = np.where(data['codigo_fc'].isin([1,6,11]), "Factura", "Credito")
data['tipo2_new'] = np.where(data['codigo_fc'] == 201, "Pyme_fc", data['tipo2_new'])
data['tipo2_new'] = np.where(data['codigo_fc'] == 203, "Pyme_nc", data['tipo2_new'])

data['tipo3_new'] = np.where(data['codigo_fc'].isin([1,3,201,203]), "A", "C")
data['tipo3_new'] = np.where(data['codigo_fc'].isin([6,8]), "B", data['tipo3_new'])

data['tipo3_new']

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Número Desde'].astype(int)

# Cliente
data['Denominación Receptor'] = data['Denominación Receptor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Cliente'] = data['Denominación Receptor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Receptor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])
data['TOTAL_10.5'] = round(data['NETO 10.5'] + data['IVA 10.5'],2)
data['TOTAL_21'] = round(data['NETO 21'] + data['IVA 21'],2)


# PONER 0 A TODAS LAS COLUMNAS QUE NO TIENEN ALGO
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 0','NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)
data['TOTAL_21'] = data['TOTAL_21'].astype(str)
data['TOTAL_10.5'] = data['TOTAL_10.5'].astype(str)

# PONER EN NEGATIVO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    mask = (data['tipo2_new'] == 'Credito') | (data['tipo2_new'] == 'Pyme_nc')
    data.loc[mask, col] = -data.loc[mask, col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','tipo2_new','tipo3_new', 'Factura',
        'Cliente','CUIT', 'TOTAL_10.5', 'TOTAL_21',
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,3237,PILISAR S A,33709180849,379961.2,30681.22,7158.85
1,01/10/2025,Factura,A,3236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,70149.06,885106.0
2,01/10/2025,Pyme_fc,A,213,PILISAR S A,33709180849,0.0,911434.46,11500025.0
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,911434.46,11500025.0
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.0,-911434.46,-11500025.0
...,...,...,...,...,...,...,...,...,...
102,30/10/2025,Factura,A,3299,PILISAR S A,33709180849,793245.19,115328.15,334119.7
103,30/10/2025,Factura,A,3297,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,107476.12,1356080.0
104,30/10/2025,Pyme_fc,A,219,NEW PGA S R L,30615265450,4059025.97,2012977.95,57270.88
105,31/10/2025,Factura,A,3300,NEW PGA S R L,30615265450,0.0,267835.4,3379413.0


Apply data transformation function to afip

In [9]:
from afip_data_transformation import transform_afip_inbound_invoices
from afip_data_transformation import transform_afip_outbound_invoices

data_inbound = pd.read_excel('../downloads/recibidos_octubre.xlsx', header=0, skiprows=1)
data_outbound = pd.read_excel('../downloads/emitidos_octubre.xlsx', header=0, skiprows=1)

afip_inbound_invoices = transform_afip_inbound_invoices(data = data_inbound)


In [10]:
afip_inbound_invoices

,Fecha,Tipo2,Tipo3,Factura,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,00009-00011663,INVERSIONES MAIPO S A,30699533412,0.0,96600.0,0.0,20286.0,0.0
1,01/10/2025,Factura,A,00008-00089201,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,474597.45
2,01/10/2025,Factura,A,00008-00089208,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1161272.0
3,01/10/2025,Factura,A,00003-00017721,FIGUEROA EDGARDO DANIEL,20165212577,0.0,32931.36,0.0,6911.5199999999995,273455.16000000003
4,01/10/2025,Factura,A,00008-00089213,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,140000.0
...,...,...,...,...,...,...,...,...,...,...,...
394,31/10/2025,Crédito,A,00003-00000124,HERNANDEZ FRANCISCO GUSTAVO Y ROSSI,30707654348,-0.0,-339917.36,-0.0,-71382.65,-0.0
395,31/10/2025,Crédito,A,00008-00008218,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,-2106948.35
396,31/10/2025,Crédito,A,00008-00008219,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,-1924116.35
397,31/10/2025,Crédito,A,00008-00008220,CARNIVAL SRL,30676579938,-0.0,-0.0,-0.0,-0.0,-2094700.95


Transform bookit report

FACTURAS DE VENTA

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

with open("../outbound.html", encoding="utf-8") as f:
    html = f.read()

# Parsear con BeautifulSoup (auto repara etiquetas mal cerradas)
soup = BeautifulSoup(html, "html.parser")

# Si no encuentra una tabla explícita, la creamos a mano
table = soup.find("table")
if not table:
    wrapped = f"<table>{soup}</table>"
    table = BeautifulSoup(wrapped, "html.parser").find("table")

# Convertir a DataFrame
df = pd.read_html(StringIO(str(table)))[0]


In [ ]:

# A PARTIR DE AHORA HAY QUE FILTRAR PARA COMPARAR CON EL DATASET DE AFIP Y ASÍ SABER QUE HAY QUE CARGAR Y QUE NO

# crear columnas de totales a utilizar
df['TOTAL_10.5'] = df['NETO 10.5'] + df['IVA 10.5']
df['TOTAL_21'] = df['NETO 21'] + df['IVA 21']
df['TOTAL_NO_GRAVADO'] = df['NO GRAVADO']

cols = ['FACTURA','TOTAL_10.5','TOTAL_21','TOTAL_NO_GRAVADO']

df = df[cols]

In [33]:
afip_outbound_invoices = transform_afip_outbound_invoices(data = data_outbound)

In [34]:
afip_outbound_invoices

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,3237,PILISAR S A,33709180849,379961.2,30681.22,7158.85
1,01/10/2025,Factura,A,3236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,70149.06,885106.0
2,01/10/2025,Pyme_fc,A,213,PILISAR S A,33709180849,0.0,911434.46,11500025.0
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,911434.46,11500025.0
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.0,-911434.46,-11500025.0
...,...,...,...,...,...,...,...,...,...
102,30/10/2025,Factura,A,3299,PILISAR S A,33709180849,793245.19,115328.15,334119.7
103,30/10/2025,Factura,A,3297,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,107476.12,1356080.0
104,30/10/2025,Pyme_fc,A,219,NEW PGA S R L,30615265450,4059025.97,2012977.95,57270.88
105,31/10/2025,Factura,A,3300,NEW PGA S R L,30615265450,0.0,267835.4,3379413.0


In [68]:
def comparar_facturas_venta(afip_df, sistem_df,tolerancia = 1.0):
    """
    Compara las facturas de AFIP con las cargadas en el sistema.
    
    Parámetros:
        afip_df (pd.DataFrame): dataset con facturas AFIP (columna NUMERO)
        sistem_df (pd.DataFrame): dataset con facturas del sistema (columna FACTURA)
        tolerancia (float): diferencia máxima aceptable en totales (default = 1.0)
    
    Devuelve:
        pd.DataFrame: copia de afip_df con columnas extra:
                      - 'loaded' (bool)
                      - 'totales_ok' (bool o NaN si no está cargada)
    """  
       # Copias para no modificar los originales
    a = afip_df.copy()
    b = sistem_df.copy()

    # --- Normalizar claves ---
    a['Factura'] = a['Factura'].astype(str).str.strip()
    b['FACTURA'] = b['FACTURA'].astype(str).str.strip()

    # --- Merge por clave ---
    merged = a.merge(
        b[['FACTURA', 'TOTAL_10.5', 'TOTAL_21', 'TOTAL_NO_GRAVADO']],
        left_on='Factura',
        right_on='FACTURA',
        how='left',
        suffixes=('', '_sistema')
    )

    # --- Columna de existencia ---
    merged['loaded'] = ~merged['FACTURA'].isna()

    # --- Forzar tipos numéricos y calcular diferencias ---
    for col in ['TOTAL_10.5', 'TOTAL_21', 'TOTAL_NO_GRAVADO']:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')
        merged[f'{col}_sistema'] = pd.to_numeric(merged[f'{col}_sistema'], errors='coerce')
        merged[f'diff_{col}'] = abs(merged[col] - merged[f'{col}_sistema'])

    # --- Evaluar si las diferencias son menores o iguales a la tolerancia ---
    merged['totales_ok'] = (
        merged[[f'diff_{c}' for c in ['TOTAL_10.5', 'TOTAL_21', 'TOTAL_NO_GRAVADO']]]
        .max(axis=1) <= tolerancia
    )

    # --- Si no está cargada, totales_ok = NaN ---
    merged.loc[~merged['loaded'], 'totales_ok'] = np.nan

    # --- Logs ---
    no_cargadas = merged[~merged['loaded']]
    mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]

    # Facturas no cargadas
    if len(no_cargadas) > 0:
        print(f"⚠️  {len(no_cargadas)} facturas no están cargadas en el sistema.")
    else:
        print("✅ Todas las facturas AFIP están cargadas en el sistema.")

    # Facturas cargadas pero con error en totales
    for factura in mal_cargadas['Factura']:
        print(f"❌ La factura {factura} no está cargada correctamente (diferencia en totales).")

    # --- Filtrar solo las facturas correctamente cargadas ---
    resultado = merged.loc[~merged['loaded'], a.columns[:9]].copy()


    print(f"✅ {len(resultado)} facturas correctamente cargadas y listas para procesar.")

    return resultado



In [69]:
solucion = comparar_facturas_venta(afip_outbound_invoices,df,1)

⚠️  19 facturas no están cargadas en el sistema.
❌ La factura 3602 no está cargada correctamente (diferencia en totales).
✅ 19 facturas correctamente cargadas y listas para procesar.


C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_21064\3895812307.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[~merged['loaded'], 'totales_ok'] = np.nan
C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_21064\3895812307.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]


In [70]:
solucion

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.00,911434.46,11500025.00
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.00,-911434.46,-11500025.00
24,08/10/2025,Factura,A,3251,NEXO LABORAL SRL,30715333348,0.00,0.00,2047044.00
26,08/10/2025,Factura,B,3590,COLEGIO DE TRADUCTORES PUBLICOS DE,30557459789,0.00,0.00,2065064.00
33,09/10/2025,Factura,B,3592,COLEGIO DE TRADUCTORES PUBLICOS DE,30557459789,0.00,0.00,1847202.00
34,09/10/2025,Factura,B,3591,VACA DANIEL ALBERTO,12888728,1574210.05,0.00,0.00
37,13/10/2025,Factura,A,3260,NEW PGA S R L,30615265450,630650.23,141762.91,537970.70
51,14/10/2025,Credito,C,1,CONSORCIO VECH,55000002185,-0.00,-5126173.02,-0.00
63,17/10/2025,Factura,B,3603,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.00,0.00,238530.56
76,21/10/2025,Factura,A,3278,NEXO LABORAL SRL,30715333348,0.00,0.00,12869375.00


FACTURAS DE COMPRA

In [88]:
with open("../inbound.html", encoding="utf-8") as f:
    html = f.read()

# Parsear con BeautifulSoup (auto repara etiquetas mal cerradas)
soup = BeautifulSoup(html, "html.parser")

# Si no encuentra una tabla explícita, la creamos a mano
table = soup.find("table")
if not table:
    wrapped = f"<table>{soup}</table>"
    table = BeautifulSoup(wrapped, "html.parser").find("table")

# Convertir a DataFrame
df = pd.read_html(StringIO(str(table)))[0]

df['TOTAL_NO_GRAVADO'] = df['NO GRAVADO']

cols = ['FACTURA','NETO 10.5','IVA 10.5','NETO 21','IVA 21','TOTAL_NO_GRAVADO']
df = df[cols]

df.head(3)

,FACTURA,NETO 10.5,IVA 10.5,NETO 21,IVA 21,TOTAL_NO_GRAVADO
0,00009-00011663,0.0,0.0,96600.0,20286.0,0.00
1,00008-00089201,0.0,0.0,0.0,0.0,474597.45
2,00008-00089208,0.0,0.0,0.0,0.0,1161272.00


In [89]:
afip_inbound_invoices.head(3)

,Fecha,Tipo2,Tipo3,Factura,Proveedor,CUIT,NETO 10.5,NETO 21,IVA 10.5,IVA 21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,00009-00011663,INVERSIONES MAIPO S A,30699533412,0.0,96600.0,0.0,20286.0,0.0
1,01/10/2025,Factura,A,00008-00089201,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,474597.45
2,01/10/2025,Factura,A,00008-00089208,CARNIVAL SRL,30676579938,0.0,0.0,0.0,0.0,1161272.0


In [91]:
cols = ['FACTURA','NETO 10.5','IVA 10.5','NETO 21','IVA 21','TOTAL_NO_GRAVADO']
df = df[cols]

In [92]:
df

,FACTURA,NETO 10.5,IVA 10.5,NETO 21,IVA 21,TOTAL_NO_GRAVADO
0,00009-00011663,0.00,0.00,96600.00,20286.00,0.00
1,00008-00089201,0.00,0.00,0.00,0.00,474597.45
2,00008-00089208,0.00,0.00,0.00,0.00,1161272.00
3,00003-00017721,0.00,0.00,32931.36,6911.52,273455.16
4,00008-00089213,0.00,0.00,0.00,0.00,140000.00
...,...,...,...,...,...,...
300,00093-00014823,0.00,0.00,270504.00,56805.84,5410.08
301,00008-00093867,0.00,0.00,0.00,0.00,6907491.00
302,00008-00093910,720006.95,75600.73,0.00,0.00,14317.73
303,00001-00000237,0.00,0.00,66115.71,13884.30,0.00


In [95]:
def comparar_facturas_compra(afip_df, sistem_df,tolerancia = 1.0):
    """
    Compara las facturas de AFIP con las cargadas en el sistema.
    
    Parámetros:
        afip_df (pd.DataFrame): dataset con facturas AFIP (columna NUMERO)
        sistem_df (pd.DataFrame): dataset con facturas del sistema (columna FACTURA)
        tolerancia (float): diferencia máxima aceptable en totales (default = 1.0)
    
    Devuelve:
        pd.DataFrame: copia de afip_df con columnas extra:
                      - 'loaded' (bool)
                      - 'totales_ok' (bool o NaN si no está cargada)
    """  
       # Copias para no modificar los originales
    a = afip_df.copy()
    b = sistem_df.copy()

    # --- Normalizar claves ---
    a['Factura'] = a['Factura'].astype(str).str.strip()
    b['FACTURA'] = b['FACTURA'].astype(str).str.strip()

    # --- Merge por clave ---
    merged = a.merge(
        b[['FACTURA', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'TOTAL_NO_GRAVADO']],
        left_on='Factura',
        right_on='FACTURA',
        how='left',
        suffixes=('', '_sistema')
    )

    # --- Columna de existencia ---
    merged['loaded'] = ~merged['FACTURA'].isna()

    # --- Forzar tipos numéricos y calcular diferencias ---
    for col in ['NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'TOTAL_NO_GRAVADO']:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')
        merged[f'{col}_sistema'] = pd.to_numeric(merged[f'{col}_sistema'], errors='coerce')
        merged[f'diff_{col}'] = abs(merged[col] - merged[f'{col}_sistema'])

    # --- Evaluar si las diferencias son menores o iguales a la tolerancia ---
    merged['totales_ok'] = merged[[f'diff_{c}' for c in ['NETO 10.5','IVA 10.5','NETO 21','IVA 21','TOTAL_NO_GRAVADO']]].max(axis=1) <= tolerancia

    # --- Si no está cargada, totales_ok = NaN ---
    merged.loc[~merged['loaded'], 'totales_ok'] = np.nan

    # --- Logs ---
    no_cargadas = merged[~merged['loaded']]
    mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]

    # Facturas no cargadas
    if len(no_cargadas) > 0:
        print(f"⚠️  {len(no_cargadas)} facturas no están cargadas en el sistema.")
    else:
        print("✅ Todas las facturas AFIP están cargadas en el sistema.")

    # Facturas cargadas pero con error en totales
    for factura in mal_cargadas['Factura']:
        print(f"❌ La factura {factura} no está cargada correctamente (diferencia en totales).")

    # --- Filtrar solo las facturas correctamente cargadas ---
    resultado = merged.loc[~merged['loaded'], a.columns[:9]].copy()


    print(f"✅ {len(resultado)} facturas correctamente cargadas y listas para procesar.")

    return resultado



In [96]:
solucion_compras = comparar_facturas_compra(afip_inbound_invoices,df,1)

⚠️  94 facturas no están cargadas en el sistema.
❌ La factura 00008-00007656 no está cargada correctamente (diferencia en totales).
✅ 94 facturas correctamente cargadas y listas para procesar.


C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_21064\1498825624.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  merged.loc[~merged['loaded'], 'totales_ok'] = np.nan
C:\Users\ThinkPad-PC\AppData\Local\Temp\ipykernel_21064\1498825624.py:49: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mal_cargadas = merged[(merged['loaded']) & (~merged['totales_ok'].fillna(False))]
